# pandas 08 - Speed up with pandas

by Nova@Douban

The video record of this session is here: https://zoom.us/recording/share/1ZCrNvlQG3rchlnczSpwCdq89ZZR12SZ75a2QvcI19WwIumekTziMw

---

In this tutotial, we will share how to process big datasets with pandas. Here, the big datasets mean those datasets are too big for a single machine.

## 8.1 Stream processing

pandas is an efficient tool to process data, but when the dataset cannot be fit in memory, using pandas could be a little bit tricky. If the dataset is big enough to take all of the memorys, the pandas task will get stuck there.

One way to deal this problem is to apply stream processing to pandas with `chunksize` parameter in  `pd.read_csv()`, `pd.read_table()`, `pd.read_json(lines=True)` 

---

### 8.1.1 An example of stream processing

In [1]:
def preprocess_patent(in_f, out_f):
    '''
    normal read and write
    '''
    df = pd.read_table(in_f, sep='##')
    df.columns = ['id0', 'id1', 'ref']
    result = df[(df['ref'].str.contains('^[a-zA-Z]+')) & (df.ref['ref'].len() > 80)]
    result.to_csv(out_f, index=False, header=False, mode='w')

def preprocess_patent(in_f, out_f, size):
    '''
    read a chunk,
    process a chunk,
    write a chunk,
    then repeat
    '''
    reader = pd.read_table(in_f, sep='##', chunksize=size)
    for chunk in reader:
        chunk.columns = ['id0', 'id1', 'ref']
        result = chunk[(chunk['ref'].str.contains('^[a-zA-Z]+')) & (chunk['ref'].str.len() > 80)]
        result.to_csv(out_f, index=False, header=False, mode='a')

Some aspects are worth paying attetion to:

1. The `chunksize` should not be too small. If it is too small, the IO cost will be high to overcome the benefit. For example, if we have a file with one million lines, we did a little experiment:

| Chunksize | Memory (MiB) | Time (s) |
|-----------|--------------|----------|
| 100       | 142.13       | 36.9     |
| 1,000     | 141.38       | 13.8     |
| 10,000    | 141.38       | 12.1     |
| 100,000   | 209.88       | 12.7     |
| 200,000   | 312.15       | 12.5     |

In our main task, we set `chunksize` as 200,000, and it used 211.22MiB memory to process the 10G+ dataset with 9min 54s.

2. the `pandas.DataFrame.to_csv()` mode should be set as 'a' to append chunk results to a single file; otherwise, only the last chunk will be saved.

### 8.1.2 Be Careful with the Index

Once, I had a strange with above stream processing logic:

In [2]:
import pandas as pd
import modin.pandas as mp

def stream_process(IN_FILE, OUT_FILE):
    reader = pd.read_csv(IN_FILE, chunksize = 1000, engine='c')
    for chunk in reader:
        result = []
        for line in chunk.tolist():
             temp = complicated_process(chunk)  # this involves a very complicated processing, so here is just a simplified version
             result.append(temp)
        chunk['new_series'] = pd.series(result)
        chunk.to_csv(OUT_TILE, index=False, mode='a')

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-01-20_20-51-34_41781/logs.
Waiting for redis server at 127.0.0.1:59136 to respond...
Waiting for redis server at 127.0.0.1:53672 to respond...
Starting the Plasma object store with 6.871947672999999 GB memory using /tmp.


---

I can confirm each loop of result is not empty. But only in the first time of the loop, line `chunk['new_series'] = pd.series(result)` has result, and the rest are empty. Therefore, only the first chunk of the output contains new_series, and the rest are empty.

When we tracked the index of each chunk, we found that they are not independent. We assumed that each chunk would start the index from 0, but in reality, it is NOT. The index of each chunk is a subset of the whole CSV in this situation, so their index derives from the CSV. This is what caused the problem. In our initial logic, the `pandas.to_csv` writes only the result of the first chunk, instead of the last chunk.

Therefore, a better solution would be rebuild index for each chunk, and concatenating it with result.

In [3]:
def stream_process(IN_FILE, OUT_FILE):
    reader = pd.read_csv(IN_FILE, chunksize = 1000, engine='c')
    for chunk in reader:
        result = []
        for line in chunk.tolist():
             temp = complicated_process(chunk)  # this involves a very complicated processing, so here is just a simplified version
             result.append(temp)
        new_chunk = chunk.reindex()  # key solver
        new_chunk = new_chunk.assign(new_series=result)
        new_chunk.to_csv(OUT_TILE, index=False, mode='a')

## 8.2 Reduce memory usage with pandas

### 8.2.1 Reduce with categorical type

Often, some columns of data are catagorical, but they are saved as non-catagorical types. Therefore, we can convert them to categorical type with `pd.Series.astype('category')`

In [4]:
def mem_usage(pandas_obj):
    '''
    We will use this to check memory usage
    '''
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

df = pd.read_csv('../data/mobile_phone.csv')
display(df.head(), df.info())
display(mem_usage(df['AREA_CODE']), mem_usage(df['AREA_CODE'].astype('category')))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321792 entries, 0 to 321791
Data columns (total 3 columns):
MSISDN_SEG    321792 non-null int64
AREA_CODE     321792 non-null int64
ASP           321792 non-null int64
dtypes: int64(3)
memory usage: 7.4 MB


,MSISDN_SEG,AREA_CODE,ASP
0,1451091,10,2
1,1451092,10,2
2,1451093,10,2
3,1451094,10,2
4,1451095,10,2


None

'2.46 MB'

'0.63 MB'

### 8.2.2 Choose the right subtypes

pandas often chooses a safer dtype to store data; however, this may additional memory usage. For example, we can downcast numerical to `unsigned int` to save space for positive-only integers.

<img src="../image/subtypes.png">


In [5]:
df_int = df.select_dtypes(include=['int'])
converted_int = df_int.apply(pd.to_numeric, downcast='unsigned')

display(df_int.head(), converted_int.info())
display(mem_usage(df_int), mem_usage(converted_int))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321792 entries, 0 to 321791
Data columns (total 3 columns):
MSISDN_SEG    321792 non-null uint32
AREA_CODE     321792 non-null uint16
ASP           321792 non-null uint8
dtypes: uint16(1), uint32(1), uint8(1)
memory usage: 2.1 MB


,MSISDN_SEG,AREA_CODE,ASP
0,1451091,10,2
1,1451092,10,2
2,1451093,10,2
3,1451094,10,2
4,1451095,10,2


None

'7.37 MB'

'2.15 MB'

### 8.2.3 Read data with clarifying dtype of each column

When we use pandas to read date, we can set `dtype` for each column, so pandas doesn't have to guess, and also saves space.

In [6]:
%time df1 = pd.read_csv('../data/mobile_phone.csv', engine='c')

column_type = {'MSISDN_SEG': 'uint32', 'AREA_CODE': 'category', 'ASP': 'category'}
%time df2 = pd.read_csv('../data/mobile_phone.csv', dtype=column_type, engine='c')

display(df1.info(), df2.info())

CPU times: user 123 ms, sys: 51.9 ms, total: 175 ms
Wall time: 299 ms
CPU times: user 111 ms, sys: 21.2 ms, total: 132 ms
Wall time: 133 ms
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321792 entries, 0 to 321791
Data columns (total 3 columns):
MSISDN_SEG    321792 non-null int64
AREA_CODE     321792 non-null int64
ASP           321792 non-null int64
dtypes: int64(3)
memory usage: 7.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321792 entries, 0 to 321791
Data columns (total 3 columns):
MSISDN_SEG    321792 non-null uint32
AREA_CODE     321792 non-null category
ASP           321792 non-null category
dtypes: category(2), uint32(1)
memory usage: 2.2 MB


None

None

## 8.3 Cython

Cython is a C implementation of Python, and pandas can work nicely with Cython. If you don't know C, you can just import Cython to gain speed; moreover, if you know C, you can modify your code to Cython syntax to gain extra speed.

In [7]:
def demo_calc(series):
    return series * 67 - 89 / 45

%time series1 = demo_calc(df1['MSISDN_SEG'])

CPU times: user 5.89 ms, sys: 2.62 ms, total: 8.51 ms
Wall time: 8.62 ms


In [8]:
%load_ext cython

%time series2 = demo_calc(df1['MSISDN_SEG'])

CPU times: user 2.09 ms, sys: 853 µs, total: 2.94 ms
Wall time: 2.95 ms


## 8.4 Modin

Modin is a DataFrame library that allows you to speed up your pandas workflows by changing one line of code. 

In [9]:
%time df1 = pd.read_csv('../data/mobile_phone.csv')
%time df2 = mp.read_csv('../data/mobile_phone.csv')

CPU times: user 89.7 ms, sys: 29.9 ms, total: 120 ms
Wall time: 118 ms
CPU times: user 8.75 ms, sys: 2.44 ms, total: 11.2 ms
Wall time: 81.7 ms


In [10]:
def demo_calc(series):
    return series * 67 - 89 / 45

def demo_calc2(series):
    return mp.Series(series) * 67 - 89 / 45

%time series1 = demo_calc(df1['MSISDN_SEG'])
%time series2 = demo_calc2(df2['MSISDN_SEG'])

CPU times: user 7.7 ms, sys: 2.33 ms, total: 10 ms
Wall time: 6.94 ms
CPU times: user 113 ms, sys: 11.6 ms, total: 124 ms
Wall time: 136 ms


---

## 8.5 Exercises

1. Read [Enhancing Performance](https://pandas.pydata.org/pandas-docs/stable/enhancingperf.html) by pandas
2. Read this post [Tutorial: Using pandas with Large Data Sets](https://www.dataquest.io/blog/pandas-big-data/)
3. We showed using `chunk_size` with `pd.read_table`, can you try this with `pd.read_json`?

---